In [1]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=53c46981876a9f3efc59d537ca4459a22f372b0a8f808eaa3ea9c2bded512acf
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [2]:
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from google.colab import files
import datetime as dt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import seaborn as sns
from sklearn.metrics import mean_squared_error
from matplotlib.gridspec import GridSpec
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE, f_regression,RFECV
import warnings
from sklearn.model_selection import StratifiedKFold,RepeatedKFold
import gc
import time
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import warnings
from bayes_opt import BayesianOptimization
import time
import sys
import matplotlib.pyplot as plt
warnings.simplefilter(action='ignore', category=FutureWarning)
import xgboost as xgb
import pickle


In [3]:
#Taken help from https://stackoverflow.com/questions/69822304/google-colab-google-drive-can%c2%b4t-be-mounted-anymore-browser-popup-google-dri
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155113 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.27-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [4]:
os.chdir('/content/drive/MyDrive/appliedai/assignment/case_study_1/real_case_study_part')

In [5]:
train = pd.read_csv('feature_engineered_train.csv')
test = pd.read_csv('feature_engineered_test.csv')

In [6]:
test_card_ids = test['card_id']

In [7]:
train.drop(['Unnamed: 0','card_id'],axis=1,inplace=True)
test.drop(['Unnamed: 0','card_id'],axis=1,inplace=True)

In [16]:
train.drop(['hist_purchase_date_max','hist_purchase_date_min'],axis=1,inplace=True)
test.drop(['hist_purchase_date_max','hist_purchase_date_min'],axis=1,inplace=True)

In [17]:
features = [c for c in train.columns if c not in ['target','outliers']]
for i in features:
  if np.any(np.isnan(train[i])):
    train[i].fillna(train[i].median(),inplace=True)

for i in features:
  if np.any(np.isnan(test[i])):
    test[i].fillna(test[i].median(),inplace=True)

# Bayesian optimization(Hyperparamer tuning)


In [11]:
# Inspired from https://www.kaggle.com/roshanm0903/dmbi-final#Ensemble-Learning
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod(
            (datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))
        

def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [12]:
def LGB_CV(
          max_depth,
          num_leaves,
          min_data_in_leaf,
          feature_fraction,
          bagging_fraction,
          lambda_l1,
          colsample_bytree
         ):
    
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    oof = np.zeros(train.shape[0])

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, train["outliers"].values)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               )
        val_data = lgb.Dataset(train.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               )
    
        param = {
            'num_leaves': int(num_leaves),
            'min_data_in_leaf': int(min_data_in_leaf), 
            'objective':'regression',
            'max_depth': int(max_depth),
            'learning_rate': 0.01,
            "boosting": "gbdt",
            "feature_fraction": feature_fraction,
            "bagging_freq": 1,
            "bagging_fraction": bagging_fraction ,
            "bagging_seed": 11,
            "metric": 'rmse',
            "lambda_l1": lambda_l1,
            "verbosity": -1,
            "colsample_bytree":colsample_bytree
        }
    
        clf = lgb.train(param,
                        trn_data,
                        10000,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=500,
                        early_stopping_rounds = 200)
        
        oof[val_idx] = clf.predict(train.iloc[val_idx][features],
                                   num_iteration=clf.best_iteration)
        
        del clf, trn_idx, val_idx
        gc.collect()
        
    return -mean_squared_error(oof, target)**0.5

train = reduce_mem_usage(train)

target = train['target']
features = [c for c in train.columns if c not in ['target','outliers']]

LGB_BO = BayesianOptimization(LGB_CV, {
    'max_depth': (4, 10),
    'num_leaves': (5, 130),
    'min_data_in_leaf': (10, 150),
    'feature_fraction': (0.7, 1.0),
    'bagging_fraction': (0.7, 1.0),
    'lambda_l1': (0, 6),
    'colsample_bytree':(0,1)
    })

print('-'*126)

start_time = timer(None)
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=2, n_iter=20, acq='ei', xi=0.0)
timer(start_time)


Mem. usage decreased to 187.75 Mb (73.0% reduction)
------------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | baggin... | colsam... | featur... | lambda_l1 | max_depth | min_da... | num_le... |
-------------------------------------------------------------------------------------------------------------
fold n°0
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.39957	valid_1's rmse: 3.68733
Early stopping, best iteration is:
[498]	training's rmse: 3.4	valid_1's rmse: 3.6873
fold n°1
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.41334	valid_1's rmse: 3.62592
[1000]	training's rmse: 3.27586	valid_1's rmse: 3.62184
Early stopping, best iteration is:
[1287]	training's rmse: 3.21237	valid_1's rmse: 3.62139
fold n°2
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 3.41837	valid_1's rm

In [13]:
LGB_BO.max['params']

{'bagging_fraction': 0.7845841218210182,
 'colsample_bytree': 0.6087224326444155,
 'feature_fraction': 0.7120470721122002,
 'lambda_l1': 2.167718400924338,
 'max_depth': 9.625131855574836,
 'min_data_in_leaf': 119.37985756172391,
 'num_leaves': 117.18296536438672}

# Model training

In [19]:
# Inspired from https://www.kaggle.com/roshanm0903/dmbi-final
param = {'num_leaves': 117,
         'min_data_in_leaf': 119, 
         'colsample_bytree': 0.6087224326444155,
         'objective':'regression',
         'max_depth': 10,
         'learning_rate': 0.005,
         "boosting": "gbdt",    
         "feature_fraction": 0.7120470721122002,
         "bagging_freq": 1,
         "bagging_fraction": 0.7845841218210182 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 2.167718400924338,
         "random_state": 133,
         "verbosity": -1}

In [20]:
features = [c for c in train.columns if c not in ['target','outliers']]
folds = StratifiedKFold(n_splits=5, shuffle = True, random_state=490)

oof = np.zeros(len(train))
predictions = np.zeros(len(test))

feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, train["outliers"].values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                           label=train["target"].iloc[trn_idx],
                             )
    val_data = lgb.Dataset(train.iloc[val_idx][features],
                           label=train["target"].iloc[val_idx],
                            )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 200)
    
    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(mean_squared_error(oof, train["target"])**0.5))


fold n°0
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.70452	valid_1's rmse: 3.75858
[200]	training's rmse: 3.62048	valid_1's rmse: 3.71921
[300]	training's rmse: 3.56325	valid_1's rmse: 3.7017
[400]	training's rmse: 3.51961	valid_1's rmse: 3.69376
[500]	training's rmse: 3.48482	valid_1's rmse: 3.6889
[600]	training's rmse: 3.4557	valid_1's rmse: 3.68617
[700]	training's rmse: 3.43001	valid_1's rmse: 3.68493
[800]	training's rmse: 3.40749	valid_1's rmse: 3.68386
[900]	training's rmse: 3.38689	valid_1's rmse: 3.68365
[1000]	training's rmse: 3.36776	valid_1's rmse: 3.68336
[1100]	training's rmse: 3.35035	valid_1's rmse: 3.68293
[1200]	training's rmse: 3.33408	valid_1's rmse: 3.68275
[1300]	training's rmse: 3.31734	valid_1's rmse: 3.68276
[1400]	training's rmse: 3.30089	valid_1's rmse: 3.68273
[1500]	training's rmse: 3.28518	valid_1's rmse: 3.68291
Early stopping, best iteration is:
[1371]	training's rmse: 3.30562	valid_1's rmse: 3.6826
fold n°1
T

In [21]:
submissions = pd.DataFrame(columns=['card_id','target'],index=list(range(test_card_ids.shape[0])))
submissions['card_id'] = test_card_ids
submissions['target'] = predictions

In [22]:
submissions.to_csv('submission.csv',index=False)

In [23]:
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
##saving the model 
pickle.dump(clf, open('best_model_lightgbm.sav', 'wb'))
